In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns 
from functools import reduce
from datetime import datetime


In [83]:
dfs.to_csv("EURUSD=X.csv",index=True)

In [87]:
dfaa = pd.read_csv("EURUSD=X.csv").set_index('Date')


In [88]:
dfaa

,High,Low,Open,EUR/USD,Volume,Adj Close
Date,,,,,,
2007-06,1.384907,1.338598,1.353601,1.367204,0,1.367204
2007-07,1.384006,1.338294,1.366998,1.364201,0,1.364201
2007-08,1.428204,1.355601,1.364201,1.427002,0,1.427002
2007-09,1.449801,1.401699,1.427104,1.447199,0,1.447199
2007-11,1.495707,1.361304,1.447011,1.469594,0,1.469594
...,...,...,...,...,...,...
2022-04,1.078516,1.035197,1.054630,1.077354,0,1.077354
2022-05,1.077006,1.038260,1.073411,1.044659,0,1.044659
2022-06,1.047790,0.995362,1.047768,1.022704,0,1.022704


In [106]:
dfs[dfs.index>'2007-04']

,High,Low,Open,EUR/USD,Volume,Adj Close
Date,,,,,,
2007-06,1.384907,1.338598,1.353601,1.367204,0,1.367204
2007-07,1.384006,1.338294,1.366998,1.364201,0,1.364201
2007-08,1.428204,1.355601,1.364201,1.427002,0,1.427002
2007-09,1.449801,1.401699,1.427104,1.447199,0,1.447199
2007-11,1.495707,1.361304,1.447011,1.469594,0,1.469594
...,...,...,...,...,...,...
2022-04,1.078516,1.035197,1.054630,1.077354,0,1.077354
2022-05,1.077006,1.038260,1.073411,1.044659,0,1.044659
2022-06,1.047790,0.995362,1.047768,1.022704,0,1.022704


In [3]:
indicators_US=['producer-price-index-yy','consumer-price-index-yy','ism-manufacturing-pmi','ism-non-manufacturing-pmi',
'consumer-confidence-index','retail-sales-yy','retail-sales-ex-autos-mm','durable-goods-orders','nonfarm-payrolls',
'industrial-production-mm','trade-balance','housing-starts',
'new-home-sales','building-permits','existing-home-sales','unemployment-rate','adp-nonfarm-employment-change',
'ny-empire-state-manufacturing-index','chicago-pmi','capacity-utilization','factory-orders','durable-goods-orders-ex-transportation',
'business-inventories-mm','pce-price-index-yy','pce-price-index-mm','core-pce-price-index-yy','core-pce-price-index-mm',
'average-hourly-earnings-mm','fed-interest-rate-decision']

indicators_EU=['producer-price-index-yy','consumer-price-index-yy',
'markit-manufacturing-pmi','markit-services-pmi','consumer-confidence-indicator',
'retail-sales-yy','industrial-production-yy','trade-balance','construction-output-yy',
'unemployment-rate','markit-composite-pmi','ecb-deposit-rate-decision','zew-indicator-of-economic-sentiment','ecb-interest-rate-decision']

indicators_interest = ['switzerland/snb-interest-rate-decision','united-kingdom/boe-interest-rate-decision']

In [6]:
len(indicators_US)+len(indicators_EU)+len(indicators_interest)

45

In [9]:
file_name = 'united-states.{}.csv'
df_list = []
for i in indicators_US:   
    df_list.append(pd.read_csv(file_name.format(i), sep='\t',header=0,names=['Date', i +' US', 'ForecastValue','PreviousValue']))

for j in  range(0,len(indicators_US)):
    df_list[j]['Date'] = pd.to_datetime(df_list[j]['Date'], format='%Y-%m').dt.strftime("%Y-%m")
    df_list[j] = df_list[j].set_index('Date').sort_values(by="Date", ascending=[0])
    df_list[j] = df_list[j][~df_list[j].index.duplicated(keep='first')]    
    

data_frames=[[]for i in range (len(indicators_US))]
for k in  range(0,len(indicators_US)):
    data_frames[k]=df_list[k][[df_list[k].columns[0]]]   
    #df = pd.concat(df_list)

# Get Forecast

In [72]:
data_frames_forecast=[[]for i in range (len(indicators_US))]
for k in  range(0,len(indicators_US)):
    data_frames_forecast[k]=df_list[k][[df_list[k].columns[1]]]
    data_frames_forecast[k] = data_frames_forecast[k].rename(columns={'ForecastValue': df_list[k].columns[0]})

    
data_frames_EuroZone_forecast=[[]for i in range (len(indicators_EU))]  
for k in  range(0,len(indicators_EU)):
    data_frames_EuroZone_forecast[k]=df_list_euro[k][[df_list_euro[k].columns[1]]]  
    data_frames_EuroZone_forecast[k] = data_frames_EuroZone_forecast[k].rename(columns={'ForecastValue': df_list_euro[k].columns[0] })
    
dfr0 = pd.read_csv("united-kingdom.boe-interest-rate-decision.csv", sep='\t')
dfr1 = pd.read_csv("switzerland.snb-interest-rate-decision.csv", sep='\t')

dfr0['Date'] = pd.to_datetime(dfr0['Date'], format='%Y-%m').dt.strftime("%Y-%m")
dfr1['Date'] = pd.to_datetime(dfr1['Date'], format='%Y-%m').dt.strftime("%Y-%m")

dfr0 = dfr0.rename(columns={'ActualValue': 'boe-interest-rate-decision UK'}).set_index('Date').sort_values(by="Date", ascending=[0])
dfr1 = dfr1.rename(columns={'ActualValue': 'snb-interest-rate-decision Switzerland'}).set_index('Date').sort_values(by="Date", ascending=[0])

dfr0 = dfr0.bfill().ffill()
dfr1 = dfr1.bfill().ffill()

    
df_merged_forecast = reduce(lambda  left,right: pd.merge(left,right,on='Date',
                                            how='outer'), data_frames_forecast)


df_merged_EuroZone_forecast = reduce(lambda  left,right: pd.merge(left,right,on='Date',
                                            how='outer'), data_frames_EuroZone)
df_merged_EuroZone_forecast = df_merged_EuroZone_forecast.sort_values(by="Date", ascending=[0])
df_merged_EuroZone_forecast [~df_merged_EuroZone_forecast.index.duplicated(keep='first')]
df_merged_EuroZone_forecast['ecb-interest-rate-decision EuroZone'] = df_merged_EuroZone_forecast['ecb-interest-rate-decision EuroZone'].bfill().ffill()


data_frames_Merged_forecast=[df_merged_forecast,df_merged_EuroZone_forecast,dfr0[['boe-interest-rate-decision UK']],dfr1[['snb-interest-rate-decision Switzerland']]]


df_forecast = reduce(lambda  left,right: pd.merge(left,right,on='Date',
                                             how='left'), data_frames_Merged_forecast)


df_forecast = df_forecast.sort_values(by="Date", ascending=[0])

df_forecast=df_forecast [~df_forecast .index.duplicated(keep='first')]
df_forecast['boe-interest-rate-decision UK'] = df_forecast['boe-interest-rate-decision UK'].bfill().ffill()
df_forecast['snb-interest-rate-decision Switzerland'] = df_forecast['snb-interest-rate-decision Switzerland'].bfill().ffill()
'''
if df_forecast[:1].isnull().sum(axis = 1).item()> 0 :
    df_forecast = df_forecast[1:len(df_list[0])-2]
else:
    df_forecast = df_forecast[0:len(df_list[0])-2]
'''

'\nif df_forecast[:1].isnull().sum(axis = 1).item()> 0 :\n    df_forecast = df_forecast[1:len(df_list[0])-2]\nelse:\n    df_forecast = df_forecast[0:len(df_list[0])-2]\n'

In [73]:
df_forecast

,producer-price-index-yy US,consumer-price-index-yy US,ism-manufacturing-pmi US,ism-non-manufacturing-pmi US,consumer-confidence-index US,retail-sales-yy US,retail-sales-ex-autos-mm US,durable-goods-orders US,nonfarm-payrolls US,industrial-production-mm US,...,industrial-production-yy EuroZone,trade-balance EuroZone,construction-output-yy EuroZone,unemployment-rate EuroZone,markit-composite-pmi EuroZone,ecb-deposit-rate-decision EuroZone,zew-indicator-of-economic-sentiment EuroZone,ecb-interest-rate-decision EuroZone,boe-interest-rate-decision UK,snb-interest-rate-decision Switzerland
Date,,,,,,,,,,,,,,,,,,,,,
2022-08,11.0,9.7,54.6,55.6,NaN,NaN,NaN,NaN,-19.0,NaN,...,2.4,NaN,NaN,6.6,49.9,NaN,NaN,0.50,1.75,-0.25
2022-07,10.0,9.0,55.8,57.7,102.4,8.2,-0.4,-8.7,-229.0,-0.2,...,1.6,-26.0,2.9,NaN,52.0,0.0,-51.1,0.50,1.25,-0.25
2022-06,11.1,8.2,56.3,57.7,106.8,7.6,0.7,0.0,-19.0,0.0,...,-2.0,-31.7,3.0,6.8,51.9,-0.5,-28.0,0.00,1.25,-0.25
2022-05,12.2,8.4,57.9,62.7,107.2,12.3,-1.4,-0.1,317.0,0.3,...,-0.8,-17.6,3.3,6.8,54.9,NaN,-29.5,0.00,1.00,-0.75
2022-04,9.4,7.9,58.2,63.4,108.8,15.7,2.1,0.1,80.0,0.3,...,2.0,-9.4,9.4,NaN,55.8,-0.5,-43.0,0.00,0.75,-0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.8,2.3,NaN,7.5,NaN,3.0,NaN,4.00,5.50,2.50
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,7.6,NaN,7.6,NaN,NaN,NaN,3.75,5.50,2.25
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,5.50,2.25


In [60]:
dfr0

,boe-interest-rate-decision UK,ForecastValue,PreviousValue
Date,,,
2022-08,1.75,0.12,1.25
2022-06,1.25,0.12,1.00
2022-05,1.00,0.12,0.75
2022-03,0.75,0.12,0.50
2022-02,0.50,0.12,0.25
...,...,...,...
2007-08,5.75,0.20,5.75
2007-07,5.75,0.20,5.50
2007-06,5.50,0.20,5.50


In [59]:
frames_forecast

[[         producer-price-index-yy US
  Date                               
  2022-08                        11.0
  2022-07                        10.0
  2022-06                        11.1
  2022-05                        12.2
  2022-04                         9.4
  ...                             ...
  2007-07                         NaN
  2007-06                         NaN
  2007-05                         NaN
  2007-03                         NaN
  2007-02                         NaN
  
  [186 rows x 1 columns],
           consumer-price-index-yy US
  Date                               
  2022-08                         9.7
  2022-07                         9.0
  2022-06                         8.2
  2022-05                         8.4
  2022-04                         7.9
  ...                             ...
  2007-06                         NaN
  2007-05                         NaN
  2007-04                         NaN
  2007-03                         NaN
  2007-02            

In [56]:
frames_forecast

[[         producer-price-index-yy US
  Date                               
  2022-08                        11.0
  2022-07                        10.0
  2022-06                        11.1
  2022-05                        12.2
  2022-04                         9.4
  ...                             ...
  2007-07                         NaN
  2007-06                         NaN
  2007-05                         NaN
  2007-03                         NaN
  2007-02                         NaN
  
  [186 rows x 1 columns],
           consumer-price-index-yy US
  Date                               
  2022-08                         9.7
  2022-07                         9.0
  2022-06                         8.2
  2022-05                         8.4
  2022-04                         7.9
  ...                             ...
  2007-06                         NaN
  2007-05                         NaN
  2007-04                         NaN
  2007-03                         NaN
  2007-02            

In [51]:
data_frames_forecast

[         producer-price-index-yy US
 Date                               
 2022-08                        11.0
 2022-07                        10.0
 2022-06                        11.1
 2022-05                        12.2
 2022-04                         9.4
 ...                             ...
 2007-07                         NaN
 2007-06                         NaN
 2007-05                         NaN
 2007-03                         NaN
 2007-02                         NaN
 
 [186 rows x 1 columns],
          consumer-price-index-yy US
 Date                               
 2022-08                         9.7
 2022-07                         9.0
 2022-06                         8.2
 2022-05                         8.4
 2022-04                         7.9
 ...                             ...
 2007-06                         NaN
 2007-05                         NaN
 2007-04                         NaN
 2007-03                         NaN
 2007-02                         NaN
 
 [187 row

In [9]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on='Date',
                                            how='outer'), data_frames)
df_merged = df_merged.sort_values(by="Date", ascending=[0])
df_merged [~df_merged .index.duplicated(keep='first')]
df_merged['fed-interest-rate-decision US'] = df_merged['fed-interest-rate-decision US'].bfill().ffill()
if df_merged[:1].isnull().sum(axis = 1).item()> 0 :
    df_merged = df_merged[1:len(df_list[0])-2]
else:
    df_merged = df_merged[0:len(df_list[0])-2]
df_merged.loc[:, df_merged.columns != 'fed-interest-rate-decision US'] = df_merged.loc[:, df_merged.columns != 'fed-interest-rate-decision US'].interpolate().bfill().ffill()



In [10]:
def missing_values_table(dataframe):
    variables_with_na = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[variables_with_na].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[variables_with_na].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df)
    return variables_with_na


missing_values_table(df_merged)

Empty DataFrame
Columns: [n_miss, ratio]
Index: []


[]

In [11]:
df_merged.dtypes

producer-price-index-yy US                   float64
consumer-price-index-yy US                   float64
ism-manufacturing-pmi US                     float64
ism-non-manufacturing-pmi US                 float64
consumer-confidence-index US                 float64
retail-sales-yy US                           float64
retail-sales-ex-autos-mm US                  float64
durable-goods-orders US                      float64
nonfarm-payrolls US                          float64
industrial-production-mm US                  float64
trade-balance US                             float64
housing-starts US                            float64
new-home-sales US                            float64
building-permits US                          float64
existing-home-sales US                       float64
unemployment-rate US                         float64
adp-nonfarm-employment-change US             float64
ny-empire-state-manufacturing-index US       float64
chicago-pmi US                               f

# Euro Zone

In [5]:
file_name_euro = 'european-union.{}.csv'
df_list_euro = []
for i in indicators_EU:   
    df_list_euro.append(pd.read_csv(file_name_euro.format(i), sep='\t',header=0,names=['Date', i +' EuroZone', 'ForecastValue','PreviousValue']))

for j in  range(0,len(indicators_EU)):
    df_list_euro[j]['Date'] = pd.to_datetime(df_list_euro[j]['Date'], format='%Y-%m').dt.strftime("%Y-%m")
    df_list_euro[j] = df_list_euro[j].set_index('Date').sort_values(by="Date", ascending=[0])
    df_list_euro[j] = df_list_euro[j][~df_list_euro[j].index.duplicated(keep='first')]    
    

data_frames_EuroZone=[[]for i in range (len(indicators_EU))]
for k in  range(0,len(indicators_EU)):
    data_frames_EuroZone[k]=df_list_euro[k][[df_list_euro[k].columns[0]]]   
    #df = pd.concat(df_list)

In [13]:
 data_frames_EuroZone

[         producer-price-index-yy EuroZone
 Date                                     
 2022-08                              35.8
 2022-07                              36.3
 2022-06                              37.2
 2022-05                              36.8
 2022-04                              31.4
 ...                                   ...
 2007-08                               2.3
 2007-07                               2.4
 2007-06                               2.4
 2007-04                               2.7
 2007-03                               2.9
 
 [185 rows x 1 columns],
          consumer-price-index-yy EuroZone
 Date                                     
 2022-07                               8.9
 2022-06                               8.1
 2022-05                               7.4
 2022-04                               7.5
 2022-03                               5.9
 ...                                   ...
 2007-07                               1.8
 2007-05                   

In [14]:
df_merged_EuroZone = reduce(lambda  left,right: pd.merge(left,right,on='Date',
                                            how='outer'), data_frames_EuroZone)
df_merged_EuroZone = df_merged_EuroZone.sort_values(by="Date", ascending=[0])
df_merged_EuroZone [~df_merged_EuroZone .index.duplicated(keep='first')]
df_merged_EuroZone['ecb-interest-rate-decision EuroZone'] = df_merged_EuroZone['ecb-interest-rate-decision EuroZone'].bfill().ffill()
if df_merged_EuroZone[:1].isnull().sum(axis = 1).item()> 0 :
    df_merged_EuroZone = df_merged_EuroZone[1:len(df_list[0])-2]
else:
    df_merged_EuroZone = df_merged_EuroZone[0:len(df_list[0])-2]
df_merged_EuroZone.loc[:, df_merged_EuroZone.columns != 'fed-interest-rate-decision US'] = df_merged_EuroZone.loc[:, df_merged_EuroZone.columns != 'fed-interest-rate-decision US'].interpolate().bfill().ffill()

In [15]:
df_merged_EuroZone

,producer-price-index-yy EuroZone,consumer-price-index-yy EuroZone,markit-manufacturing-pmi EuroZone,markit-services-pmi EuroZone,consumer-confidence-indicator EuroZone,retail-sales-yy EuroZone,industrial-production-yy EuroZone,trade-balance EuroZone,construction-output-yy EuroZone,unemployment-rate EuroZone,markit-composite-pmi EuroZone,ecb-deposit-rate-decision EuroZone,zew-indicator-of-economic-sentiment EuroZone,ecb-interest-rate-decision EuroZone
Date,,,,,,,,,,,,,,
2022-07,36.3,8.90,52.1,53.0,-27.0,0.2,1.6,-26.0,2.9,6.80,52.0,0.000000,-51.1,0.50
2022-06,37.2,8.10,52.0,52.8,-23.6,3.9,-2.0,-31.7,3.0,6.80,51.9,-0.500000,-28.0,0.00
2022-05,36.8,7.40,54.4,56.3,-21.1,0.8,-0.8,-17.6,3.3,6.80,54.9,-0.500000,-29.5,0.00
2022-04,31.4,7.50,55.3,57.7,-16.9,5.0,2.0,-9.4,9.4,6.80,55.8,-0.500000,-43.0,0.00
2022-03,30.6,5.90,57.0,54.8,-18.7,7.8,-1.3,-7.7,4.1,6.80,54.5,-0.500000,-38.7,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-09,1.8,1.70,54.3,54.0,-6.0,1.3,3.9,0.8,0.9,7.35,46.0,3.057692,-20.3,4.00
2007-08,2.3,1.80,54.2,57.9,-4.0,1.0,2.4,4.4,0.9,7.40,46.0,3.038462,-20.3,4.00
2007-07,2.4,1.80,54.8,58.1,-2.0,0.1,2.5,3.9,0.9,7.50,46.0,3.019231,-20.3,4.00


# Eur/USD

In [16]:
import pandas_datareader.data as web

dfs= web.get_data_yahoo('EURUSD=X','06/01/2007',interval='m')
dfs = dfs.iloc[::1]
dfs.index =  pd.to_datetime(dfs.index, format='%Y-%m').strftime("%Y-%m")
dfs = dfs.rename(columns={'Close': 'EUR/USD'})


In [17]:
def missing_values_table(dataframe):
    variables_with_na = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[variables_with_na].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[variables_with_na].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df)
    return variables_with_na


missing_values_table(dfs)

Empty DataFrame
Columns: [n_miss, ratio]
Index: []


[]

In [18]:
dfs=dfs.sort_values(by="Date", ascending=[0])

In [107]:
import pickle
import bz2
with open('EURUSD=X.pkl', 'wb') as f:
    pickle.dump(dfs, f)

In [109]:
with open('EURUSD=X.pkl', 'rb') as f:
    dfs = pickle.load(f)

In [19]:
dfs.dtypes

High         float64
Low          float64
Open         float64
EUR/USD      float64
Volume         int64
Adj Close    float64
dtype: object

In [20]:
dfs

,High,Low,Open,EUR/USD,Volume,Adj Close
Date,,,,,,
2022-08,1.017605,1.012863,1.016363,1.013377,0,1.013377
2022-07,1.036914,1.012320,1.020825,1.025641,0,1.025641
2022-06,1.047790,0.995362,1.047768,1.022704,0,1.022704
2022-05,1.077006,1.038260,1.073411,1.044659,0,1.044659
2022-04,1.078516,1.035197,1.054630,1.077354,0,1.077354
...,...,...,...,...,...,...
2007-09,1.449801,1.401699,1.427104,1.447199,0,1.447199
2007-08,1.428204,1.355601,1.364201,1.427002,0,1.427002
2007-07,1.384006,1.338294,1.366998,1.364201,0,1.364201


In [103]:
dfr0 =read_csv('EURUSD=X.csv')

TypeError: argument of type 'float' is not iterable

In [ ]:
dfr0

In [69]:
dfr0.dtypes

High         object
Low          object
Open         object
EUR/USD      object
Volume       object
Adj Close    object
dtype: object

# Interest

In [21]:
dfr0 = pd.read_csv("united-kingdom.boe-interest-rate-decision.csv", sep='\t')
dfr1 = pd.read_csv("switzerland.snb-interest-rate-decision.csv", sep='\t')

dfr0['Date'] = pd.to_datetime(dfr0['Date'], format='%Y-%m').dt.strftime("%Y-%m")
dfr1['Date'] = pd.to_datetime(dfr1['Date'], format='%Y-%m').dt.strftime("%Y-%m")

dfr0 = dfr0.rename(columns={'ActualValue': 'boe-interest-rate-decision UK'}).set_index('Date').sort_values(by="Date", ascending=[0])
dfr1 = dfr1.rename(columns={'ActualValue': 'snb-interest-rate-decision Switzerland'}).set_index('Date').sort_values(by="Date", ascending=[0])


In [22]:
dfr0 = dfr0.bfill().ffill()
dfr1 = dfr1.bfill().ffill()

In [23]:
df_list[0]

,producer-price-index-yy US,ForecastValue,PreviousValue
Date,,,
2022-08,9.8,11.0,11.3
2022-07,11.3,10.0,10.8
2022-06,10.8,11.1,11.0
2022-05,11.0,12.2,11.2
2022-04,11.2,9.4,10.0
...,...,...,...
2007-07,3.3,NaN,4.1
2007-06,4.1,NaN,3.2
2007-05,3.2,NaN,3.2


In [269]:
dfs

,High,Low,Open,EUR/USD,Volume,Adj Close
Date,,,,,,
2022-08,1.027116,1.018952,1.026273,1.021033,0,1.021033
2022-07,1.036914,1.012320,1.020825,1.031672,0,1.031672
2022-06,1.047790,0.995362,1.047768,1.022704,0,1.022704
2022-05,1.077006,1.038260,1.073411,1.044659,0,1.044659
2022-04,1.078516,1.035197,1.054630,1.077354,0,1.077354
...,...,...,...,...,...,...
2007-09,1.449801,1.401699,1.427104,1.447199,0,1.447199
2007-08,1.428204,1.355601,1.364201,1.427002,0,1.427002
2007-07,1.384006,1.338294,1.366998,1.364201,0,1.364201


# Merging data

In [24]:
data_frames_Merged=[df_merged,df_merged_EuroZone,dfr0[['boe-interest-rate-decision UK']],dfr1[['snb-interest-rate-decision Switzerland']],dfs[['EUR/USD']]]

In [25]:
df = reduce(lambda  left,right: pd.merge(left,right,on='Date',
                                             how='left'), data_frames_Merged)


df = df.sort_values(by="Date", ascending=[0])
df=df [~df .index.duplicated(keep='first')]
df['boe-interest-rate-decision UK'] = df['boe-interest-rate-decision UK'].bfill().ffill()
df['snb-interest-rate-decision Switzerland'] = df['snb-interest-rate-decision Switzerland'].bfill().ffill()
if df[:1].isnull().sum(axis = 1).item()> 0 :
    df = df[1:len(df_list[0])-2]
else:
    df = df[0:len(df_list[0])-2]


In [26]:
df

,producer-price-index-yy US,consumer-price-index-yy US,ism-manufacturing-pmi US,ism-non-manufacturing-pmi US,consumer-confidence-index US,retail-sales-yy US,retail-sales-ex-autos-mm US,durable-goods-orders US,nonfarm-payrolls US,industrial-production-mm US,...,trade-balance EuroZone,construction-output-yy EuroZone,unemployment-rate EuroZone,markit-composite-pmi EuroZone,ecb-deposit-rate-decision EuroZone,zew-indicator-of-economic-sentiment EuroZone,ecb-interest-rate-decision EuroZone,boe-interest-rate-decision UK,snb-interest-rate-decision Switzerland,EUR/USD
Date,,,,,,,,,,,,,,,,,,,,,
2022-07,11.3,9.1,53.0,55.3,95.7,8.5,1.0,1.9,372.0,-0.2,...,-26.0,2.9,6.80,52.0,0.000000,-51.1,0.50,1.25,-0.25,1.025641
2022-06,10.8,8.6,56.1,55.9,98.7,8.1,0.5,0.7,390.0,0.2,...,-31.7,3.0,6.80,51.9,-0.500000,-28.0,0.00,1.25,-0.25,1.022704
2022-05,11.0,8.3,55.4,57.1,106.4,8.2,0.6,0.4,428.0,1.1,...,-17.6,3.3,6.80,54.9,-0.500000,-29.5,0.00,1.00,-0.75,1.044659
2022-04,11.2,8.5,57.1,58.3,107.3,6.9,1.1,0.8,431.0,0.9,...,-9.4,9.4,6.80,55.8,-0.500000,-43.0,0.00,0.75,-0.75,1.077354
2022-03,10.0,7.9,58.6,56.5,107.2,17.6,0.2,-2.2,678.0,0.5,...,-7.7,4.1,6.80,54.5,-0.500000,-38.7,0.00,0.75,-0.75,1.054630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-09,2.2,2.0,52.9,55.8,99.5,3.7,-0.4,-5.3,93.0,0.0,...,0.8,0.9,7.35,46.0,3.057692,-20.3,4.00,5.75,2.75,1.447199
2007-08,4.0,2.4,53.8,55.8,105.0,3.2,0.7,6.1,68.0,0.5,...,4.4,0.9,7.40,46.0,3.038462,-20.3,4.00,5.75,2.50,1.427002
2007-07,3.3,2.7,56.0,60.7,111.9,3.8,-0.4,1.9,69.0,0.6,...,3.9,0.9,7.50,46.0,3.019231,-20.3,4.00,5.75,2.50,1.364201


In [29]:
def missing_values_table(dataframe):
    variables_with_na = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[variables_with_na].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[variables_with_na].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df)
    return variables_with_na


missing_values_table(df)

Empty DataFrame
Columns: [n_miss, ratio]
Index: []


[]

In [30]:
df.dtypes

producer-price-index-yy US                      float64
consumer-price-index-yy US                      float64
ism-manufacturing-pmi US                        float64
ism-non-manufacturing-pmi US                    float64
consumer-confidence-index US                    float64
retail-sales-yy US                              float64
retail-sales-ex-autos-mm US                     float64
durable-goods-orders US                         float64
nonfarm-payrolls US                             float64
industrial-production-mm US                     float64
trade-balance US                                float64
housing-starts US                               float64
new-home-sales US                               float64
building-permits US                             float64
existing-home-sales US                          float64
unemployment-rate US                            float64
adp-nonfarm-employment-change US                float64
ny-empire-state-manufacturing-index US          

In [28]:
df['EUR/USD'] = dfs['EUR/USD'][1:].values

In [31]:
df.to_csv('macro_data_final2.csv',index=True)

In [34]:
df=pd.read_csv('macro_data_final2.csv')

In [36]:
df

,Date,producer-price-index-yy US,consumer-price-index-yy US,ism-manufacturing-pmi US,ism-non-manufacturing-pmi US,consumer-confidence-index US,retail-sales-yy US,retail-sales-ex-autos-mm US,durable-goods-orders US,nonfarm-payrolls US,...,trade-balance EuroZone,construction-output-yy EuroZone,unemployment-rate EuroZone,markit-composite-pmi EuroZone,ecb-deposit-rate-decision EuroZone,zew-indicator-of-economic-sentiment EuroZone,ecb-interest-rate-decision EuroZone,boe-interest-rate-decision UK,snb-interest-rate-decision Switzerland,EUR/USD
0,2022-07,11.3,9.1,53.0,55.3,95.7,8.5,1.0,1.9,372.0,...,-26.0,2.9,6.80,52.0,0.000000,-51.1,0.50,1.25,-0.25,1.025641
1,2022-06,10.8,8.6,56.1,55.9,98.7,8.1,0.5,0.7,390.0,...,-31.7,3.0,6.80,51.9,-0.500000,-28.0,0.00,1.25,-0.25,1.022704
2,2022-05,11.0,8.3,55.4,57.1,106.4,8.2,0.6,0.4,428.0,...,-17.6,3.3,6.80,54.9,-0.500000,-29.5,0.00,1.00,-0.75,1.044659
3,2022-04,11.2,8.5,57.1,58.3,107.3,6.9,1.1,0.8,431.0,...,-9.4,9.4,6.80,55.8,-0.500000,-43.0,0.00,0.75,-0.75,1.077354
4,2022-03,10.0,7.9,58.6,56.5,107.2,17.6,0.2,-2.2,678.0,...,-7.7,4.1,6.80,54.5,-0.500000,-38.7,0.00,0.75,-0.75,1.054630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2007-09,2.2,2.0,52.9,55.8,99.5,3.7,-0.4,-5.3,93.0,...,0.8,0.9,7.35,46.0,3.057692,-20.3,4.00,5.75,2.75,1.447199
179,2007-08,4.0,2.4,53.8,55.8,105.0,3.2,0.7,6.1,68.0,...,4.4,0.9,7.40,46.0,3.038462,-20.3,4.00,5.75,2.50,1.427002
180,2007-07,3.3,2.7,56.0,60.7,111.9,3.8,-0.4,1.9,69.0,...,3.9,0.9,7.50,46.0,3.019231,-20.3,4.00,5.75,2.50,1.364201
181,2007-06,4.1,2.7,55.0,59.7,105.3,5.0,-0.4,-2.3,190.0,...,2.3,0.9,7.50,46.0,3.000000,-20.3,4.00,5.50,2.50,1.367204


In [92]:
df_list[0].columns = 

In [112]:
df_list[0][:1], df_list[0].columns = df_list[0].columns , ['Date','Actual','Forecast','Previous']

In [96]:
df.drop(df.columns[0], axis=1,inplace=True)

In [111]:
df_list[0].drop(df_list[0].columns[0], axis=1,inplace=True)

In [113]:
df_list[0]

,Date,Actual,Forecast,Previous
0,Jun 2022,11.3%,10.0%,Unnamed: 4
1,Apr 2022,11.0%,12.2%,NaN
2,Mar 2022,11.2%,9.4%,NaN
3,Feb 2022,10.0%,9.1%,NaN
4,Jan 2022,9.7%,10.1%,NaN
...,...,...,...,...
9285,Jun 2007,3.3%,NaN,NaN
9286,May 2007,4.1%,NaN,NaN
9287,Apr 2007,3.2%,NaN,NaN
9288,Feb 2007,3.2%,NaN,NaN


In [24]:
df = pd.DataFrame(columns = ['0','Date','Actual','Forecast','Previous'])

In [25]:
df

,0,Date,Actual,Forecast,Previous


In [142]:
df.to_csv('./data' + 'test' + '.csv', mode='a' )

In [143]:
dfa=pd.read_csv('datatest.csv',names=['0','Date','Actual','Forecast','Previous'])

In [144]:
dfa

,0,Date,Actual,Forecast,Previous
NaN,0,Jun 2022,11.3%,10.0%,Unnamed: 4
0.0,1.0,May 2022,10.8%,11.1%,NaN
1.0,2.0,Apr 2022,11.0%,12.2%,NaN
2.0,3.0,Mar 2022,11.2%,9.4%,NaN
3.0,4.0,Feb 2022,10.0%,9.1%,NaN
...,...,...,...,...,...
9472.0,181.0,Jun 2007,3.3%,NaN,NaN
9473.0,182.0,May 2007,4.1%,NaN,NaN
9474.0,183.0,Apr 2007,3.2%,NaN,NaN
9475.0,184.0,Feb 2007,3.2%,NaN,NaN


In [38]:
df0 = pd.read_csv("united-states.producer-price-index-yy.csv", sep='\t')
df1 = pd.read_csv("united-states.consumer-price-index-yy.csv", sep='\t')
df2 = pd.read_csv("united-states.ism-manufacturing-pmi.csv", sep='\t')
df3 = pd.read_csv("united-states.ism-non-manufacturing-pmi.csv", sep='\t')
df4 = pd.read_csv("united-states.consumer-confidence-index.csv", sep='\t')
df5 = pd.read_csv("united-states.nfib-small-business-optimism.csv")
df6 = pd.read_csv("united-states.retail-sales-yy.csv", sep='\t')
df7 = pd.read_csv("united-states.retail-sales-ex-autos-mm.csv", sep='\t')
df8 = pd.read_csv("united-states.durable-goods-orders.csv", sep='\t')
df9 = pd.read_csv("united-states.nonfarm-payrolls.csv", sep='\t')
df10 = pd.read_csv("united-states.industrial-production-mm.csv", sep='\t')
df11 = pd.read_csv("united-states.trade-balance.csv", sep='\t')
df12 = pd.read_csv("united-states.housing-starts.csv", sep='\t')
df13 = pd.read_csv("united-states.new-home-sales.csv", sep='\t')
df14 = pd.read_csv("united-states.building-permits.csv", sep='\t')
df15 = pd.read_csv("united-states.existing-home-sales.csv", sep='\t')
df16 = pd.read_csv("united-states.unemployment-rate.csv", sep='\t')

df17 = pd.read_csv("united-states.adp-nonfarm-employment-change.csv", sep='\t')
df18 = pd.read_csv("united-states.ny-empire-state-manufacturing-index.csv", sep='\t')
df19 = pd.read_csv("united-states.chicago-pmi.csv", sep='\t')
df20 = pd.read_csv("united-states.capacity-utilization.csv", sep='\t')
df21 = pd.read_csv("united-states.factory-orders.csv", sep='\t')
df22 = pd.read_csv("united-states.durable-goods-orders-ex-transportation.csv", sep='\t')
df23 = pd.read_csv("united-states.business-inventories-mm.csv", sep='\t')
df24 = pd.read_csv("united-states.pce-price-index-yy.csv", sep='\t')
df25 = pd.read_csv("united-states.pce-price-index-mm.csv", sep='\t')
df26 = pd.read_csv("united-states.core-pce-price-index-yy.csv", sep='\t')
df27 = pd.read_csv("united-states.core-pce-price-index-mm.csv", sep='\t')
df28 = pd.read_csv("united-states.average-hourly-earnings-mm.csv", sep='\t')
df29 = pd.read_csv("united-states.adp-nonfarm-employment-change.csv", sep='\t')
df30 = pd.read_csv("united-states.fed-interest-rate-decision.csv", sep='\t')

In [39]:
df0

,Date,ActualValue,ForecastValue,PreviousValue
0,2022.08.11,9.8,11.0,11.3
1,2022.07.14,11.3,10.0,10.8
2,2022.06.14,10.8,11.1,11.0
3,2022.05.12,11.0,12.2,11.2
4,2022.04.13,11.2,9.4,10.0
...,...,...,...,...
181,2007.07.17,3.3,NaN,4.1
182,2007.06.14,4.1,NaN,3.2
183,2007.05.11,3.2,NaN,3.2
184,2007.03.15,3.2,NaN,0.2
